<a href="https://colab.research.google.com/github/hmh100298/keras_tuner/blob/main/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import tensorflow as tf
from tensorflow import keras


In [68]:
!pip install keras-tuner --upgrade


In [69]:
import keras_tuner as kt
from keras_tuner.tuners import Hyperband

(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()  ### du lieu quan ao tu fashion mnist
# chuan hoa gia tri pixel ve [0,1]
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [70]:
# hyper_model: build a model for hypertuning, hypertuning: quá trình lựa chọn quyền thiết lập các siêu tham số cho việc học máy tính của bạn ứng dụng (ML) 
#The model you set up for hypertuning is called a hypermodel.
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28))) #flatten : lam phang
# Tune the number of units in the first Dense layer
# Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10)) # 10 neural
# Tune the learning rate for the optimizer
# Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), # configure model
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])


In [71]:
#To instantiate the Hyperband tuner, you must specify the hypermodel, the objective to optimize and the maximum number of epochs to train (max_epochs).
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')
# Create a callback to stop training early after reaching a certain value for the validation loss.
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# Run the hyperparameter search. 
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

RuntimeError: ignored